# 1. Install The Object Detection Package

In [1]:
import shutil
import os
tmpModelPath ='/kaggle/working/models'
if os.path.exists(tmpModelPath) and os.path.isdir(tmpModelPath):
  shutil.rmtree(tmpModelPath)

MLENVIRONMENT="KAGGLE"
!git clone --depth 1 https://github.com/tensorflow/models
!cd models && git fetch --depth 1 origin ad1f7b56943998864db8f5db0706950e93bb7d81 && git checkout ad1f7b56943998864db8f5db0706950e93bb7d81
!pip install protobuf==3.20.3

Cloning into 'models'...
remote: Enumerating objects: 4326, done.
remote: Counting objects: 100% (4326/4326), done.
remote: Compressing objects: 100% (3345/3345), done.
remote: Total 4326 (delta 1208), reused 2043 (delta 909), pack-reused 0 (from 0)
Receiving objects: 100% (4326/4326), 53.64 MiB | 9.31 MiB/s, done.
Resolving deltas: 100% (1208/1208), done.
remote: Enumerating objects: 3089, done.
remote: Counting objects: 100% (3089/3089), done.
remote: Compressing objects: 100% (1379/1379), done.
remote: Total 1844 (delta 1238), reused 702 (delta 446), pack-reused 0 (from 0)
Receiving objects: 100% (1844/1844), 10.07 MiB | 20.42 MiB/s, done.
Resolving deltas: 100% (1238/1238), completed with 753 local objects.
From https://github.com/tensorflow/models
 * branch            ad1f7b56943998864db8f5db0706950e93bb7d81 -> FETCH_HEAD
Note: switching to 'ad1f7b56943998864db8f5db0706950e93bb7d81'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them,

In [2]:
# Environment Setup
import os
import sys
import re

print(sys.version)
if(MLENVIRONMENT == "KAGGLE"):
    print(" env setup")
    os.environ["HOMEFOLDER"] = "/kaggle/working/"
    HOMEFOLDER = '{HOMEFOLDER}'.format(**os.environ)
    FINALOUTPUTFOLDER_DIRNAME = '/kaggle/output/final_output'
    FINALOUTPUTFOLDER = '/kaggle/output'
    print(HOMEFOLDER)

# Copy setup files into models/research folder
!cd {HOMEFOLDER}models/research && pwd && protoc object_detection/protos/*.proto --python_out=.

# Modify setup.py
with open(HOMEFOLDER+'models/research/object_detection/packages/tf2/setup.py') as f:
    s = f.read()

with open(HOMEFOLDER+'models/research/setup.py', 'w') as f:
    if(MLENVIRONMENT == "KAGGLE"):
        s = re.sub('tf-models-official>=2.5.1','tf-models-official==2.15.0', s)
        f.write(s)

3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]
 env setup
/kaggle/working/
/kaggle/working/models/research


In [3]:
# Install
!pip install {HOMEFOLDER}models/research/
if(MLENVIRONMENT == "KAGGLE"):
    !pip install tensorflow==2.15.0
    !pip install protobuf==3.20.3

Processing ./models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 61.

Test the environment by running `model_builder_tf2_test.py` to make sure everything is working as expected.

In [4]:
!python {HOMEFOLDER}models/research/object_detection/builders/model_builder_tf2_test.py

2025-04-01 19:01:49.306788: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-01 19:01:49.306856: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-01 19:01:49.308657: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Running tests under Python 3.10.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
W0401 19:01:56.866208 134065620653184 batch_normalization.py:1531] `tf.keras.layers.experimental.SyncBatchNormalization` endpoint is deprecated and will be removed in a future release. Please use `tf.keras.layers.BatchNormalization` with parameter 

# 1.1. Get Dataset From Google Drive

1. Expand this section
2. Upload your RoboFlow .tfrecord.zip to Google Drive
3. Share the uploaded .tfrecord.zip such that anyone with the link can access the file.
4. Run this block
5. Paste your Google Drive file share link into the text box that appears after running this block
6. Click the "Process Dataset" Buttton
7. Click the Refresh button in the "Files" pane to ensure dataset.zip exists

In [5]:
import gdown
import os

def download_dataset():
    try:
        print("Downloading dataset...")
        url = 'https://drive.google.com/file/d/1Kq1w6e60OsyS13zUtYLdEWUoBQ9fZWIe/view?usp=drive_link'
        output = '/kaggle/working/dataset.zip'
        gdown.download(url, output, fuzzy=True)
        print("Download complete!")
    except Exception as e:
        print(f"Error: {str(e)}")

# Install gdown if not already installed
!pip install -q gdown --upgrade

# Execute
download_dataset()

Downloading...
From (original): https://drive.google.com/uc?id=1Kq1w6e60OsyS13zUtYLdEWUoBQ9fZWIe
From (redirected): https://drive.google.com/uc?id=1Kq1w6e60OsyS13zUtYLdEWUoBQ9fZWIe&confirm=t&uuid=5dc7b76a-166d-4235-b0ea-3626aab88872
To: /kaggle/working/dataset.zip
100%|██████████| 79.7M/79.7M [00:00<00:00, 187MB/s]

Download complete!


# 2. Auto-detect relevant tfrecord components

In [6]:
datasetPath = '/kaggle/working/dataset.zip'
print(datasetPath)
!unzip $datasetPath

/kaggle/working/dataset.zip
Archive:  /kaggle/working/dataset.zip
  inflating: README.dataset.txt      
  inflating: README.roboflow.txt     
   creating: test/
 extracting: test/Samples.tfrecord   
  inflating: test/Samples_label_map.pbtxt  
   creating: train/
 extracting: train/Samples.tfrecord  
  inflating: train/Samples_label_map.pbtxt  
   creating: valid/
 extracting: valid/Samples.tfrecord  
  inflating: valid/Samples_label_map.pbtxt  
  inflating: negative/blank.tfrecord  
 extracting: negative/blank_label_map.pbtxt  


In [7]:
import os
import fnmatch

def find_files(directory, pattern):
    """Find files matching a pattern in a directory and its subdirectories."""
    for root, dirs, files in os.walk(directory):
        for filename in fnmatch.filter(files, pattern):
            yield os.path.join(root, filename)

def set_tfrecord_variables(directory):
    train_record_fname = ''
    val_record_fname = ''
    negative_record_fname = ''
    label_map_pbtxt_fname = ''

    for tfrecord_file in find_files(directory, '*.tfrecord'):
        if '/train/' in tfrecord_file:
            train_record_fname = tfrecord_file
        elif '/valid/' in tfrecord_file:
            val_record_fname = tfrecord_file
        elif '/negative/' in tfrecord_file:
            negative_record_fname = tfrecord_file
        elif '/test/' in tfrecord_file:
            pass

    # 使用自定义的label map文件
    for label_map_file in find_files(directory, '*_label_map.pbtxt'):
        if '/train/' in label_map_file:  # 优先使用train目录下的label map
            label_map_pbtxt_fname = label_map_file
            break
        label_map_pbtxt_fname = label_map_file  # 如果找不到train目录下的，就用找到的第一个

    return train_record_fname, val_record_fname, negative_record_fname, label_map_pbtxt_fname


# Update the function call
train_record_fname, val_record_fname, negative_record_fname, label_map_pbtxt_fname = set_tfrecord_variables('/kaggle/working')

print("Train Record File:", train_record_fname)
print("Validation Record File:", val_record_fname)
print("Negative Samples Record File:", negative_record_fname)
print("Label Map File:", label_map_pbtxt_fname)

Train Record File: /kaggle/working/train/Samples.tfrecord
Validation Record File: /kaggle/working/valid/Samples.tfrecord
Negative Samples Record File: /kaggle/working/negative/blank.tfrecord
Label Map File: /kaggle/working/train/Samples_label_map.pbtxt


# 3.&nbsp;Training Configuration and Labels File Generation

Download the pre-trained Limelight Base Model

In [8]:
chosen_model = 'ssd-mobilenet-v2'
MODELS_CONFIG = {
    'ssd-mobilenet-v2': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'limelight_ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'limelight_ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
    },
}
model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

# Create "mymodel" folder for pre-trained weights and configuration files
%cd ~
%mkdir {HOMEFOLDER}models/mymodel/
%cd {HOMEFOLDER}models/mymodel/
%pwd

# Download pre-trained model weights
import tarfile
download_tar = 'https://downloads.limelightvision.io/models/' + pretrained_checkpoint
!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

# Download training configuration file for model
download_config = 'https://downloads.limelightvision.io/models/' + base_pipeline_file
!wget {download_config}
%cd ~

# Set training parameters for the model
num_steps = 40000
checkpoint_every = 2000
batch_size = 16


/root
/kaggle/working/models/mymodel
--2025-04-01 19:02:41--  https://downloads.limelightvision.io/models/limelight_ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
Resolving downloads.limelightvision.io (downloads.limelightvision.io)... 52.84.162.6, 52.84.162.60, 52.84.162.104, ...
Connecting to downloads.limelightvision.io (downloads.limelightvision.io)|52.84.162.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46042990 (44M) [application/x-gzip]
Saving to: ‘limelight_ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz’

limelight_ssd_mobil 100%[===================>]  43.91M   177MB/s    in 0.2s    

2025-04-01 19:02:41 (177 MB/s) - ‘limelight_ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz’ saved [46042990/46042990]

--2025-04-01 19:02:42--  https://downloads.limelightvision.io/models/limelight_ssd_mobilenet_v2_320x320_coco17_tpu-8.config
Resolving downloads.limelightvision.io (downloads.limelightvision.io)... 52.84.162.60, 52.84.162.6, 52.84.162.104, ...
Connecting to 

Generate Labels File

In [9]:

# Set file locations and get number of classes for config file
pipeline_fname = HOMEFOLDER+'models/mymodel/' + base_pipeline_file
fine_tune_checkpoint = HOMEFOLDER+'models/mymodel/' + model_name + '/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

def get_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)

    class_names = [category['name'] for category in category_index.values()]
    return class_names

def create_label_file(filename, labels):
    with open(filename, 'w') as file:
        for label in labels:
            file.write(label + '\n')


num_classes = get_num_classes(label_map_pbtxt_fname)
classes = get_classes(label_map_pbtxt_fname)

print('Total classes:', num_classes)
print(classes)


#Generate labels file
create_label_file(HOMEFOLDER + "limelight_neural_detector_labels.txt", classes)

Total classes: 3
['blue sample', 'red sample', 'yellow sample']


Modify the base Limelight Model Configuration File

Augmentation Options: https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto

In [10]:
# Create custom configuration file by writing the dataset, model checkpoint, and training parameters into the base pipeline file
import re

print('writing custom configuration file')



with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:

    # Set fine_tune_checkpoint path
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)

    # Set tfrecord files for train and test datasets
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(val_record_fname), s)
    s = re.sub(
        'input_path: "{}"'.format(train_record_fname),
        'input_path: "{}"\n      input_path: "{}"'.format(train_record_fname, negative_record_fname),
        s)

    # Set label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set batch_size
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)

    # Set number of classes num_classes
    s = re.sub('checkpoint_every_n: [0-9]+',
               'num_classes: {}'.format(num_classes), s)

    # Change fine-tune checkpoint type from "classification" to "detection"
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)

    # If using ssd-mobilenet-v2, reduce learning rate
    if chosen_model == 'ssd-mobilenet-v2':
      s = re.sub('learning_rate_base: .8',
                 'learning_rate_base: .004', s)

      s = re.sub('warmup_learning_rate: 0.13333',
                 'warmup_learning_rate: .0016666', s)

    # If using efficientdet-d0, use fixed_shape_resizer instead of keep_aspect_ratio_resizer (because it isn't supported by TFLite)
    if chosen_model == 'efficientdet-d0':
      s = re.sub('keep_aspect_ratio_resizer', 'fixed_shape_resizer', s)
      s = re.sub('pad_to_max_dimension: true', '', s)
      s = re.sub('min_dimension', 'height', s)
      s = re.sub('max_dimension', 'width', s)

    f.write(s)

# (Optional) Display the custom configuration file's contents
# !cat pipeline_file.config
# Set the path to the custom config file and the directory to store training checkpoints in
pipeline_file = 'pipeline_file.config'
model_dir = HOMEFOLDER+'training_progress/'
print(" ")
print(model_dir)

writing custom configuration file
 
/kaggle/working/training_progress/


# 4.&nbsp;Train Model

Once training starts, come back and click the refresh button within the tensorboard window to check training progress.



In [11]:
%load_ext tensorboard
%tensorboard --logdir '/kaggle/working/training_progress/train'

<IPython.core.display.Javascript object>

Fix TF 2.15 breaking changes

In [12]:
!pip install tf_slim --upgrade


In [13]:
import shutil
import re

original_path = '/usr/local/lib/python3.10/dist-packages/tf_slim/data/tfexample_decoder.py'
with open(original_path, 'r') as file:
  content = file.read()
  content = re.sub(r'import abc', 'import tensorflow as tf\n\nimport abc', content)
  content = re.sub(r'control_flow_ops.case', 'tf.case', content)
  content = re.sub(r'control_flow_ops.cond', 'tf.compat.v1.cond', content)
with open(original_path, 'w') as file:
  file.write(content)

print(f"File {original_path} fixed.")

File /usr/local/lib/python3.10/dist-packages/tf_slim/data/tfexample_decoder.py fixed.


Train

In [14]:
!rm -rf {HOMEFOLDER}training_progress
# Run training!
!python {HOMEFOLDER}models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --checkpoint_every_n={checkpoint_every} \
    --num_train_steps={num_steps} \
    --num_workers=2 \
    --sample_1_of_n_eval_examples=1

2025-04-01 19:02:55.575282: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-01 19:02:55.575334: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-01 19:02:55.576808: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0401 19:02:59.517235 134477283476608 deprecation.py:50] From /kaggle/working/models/research/object_detection/model_main_tf2.py:100: _CollectiveAllReduceStrategyExperimental.__init__ (from tensorflow.python.distribute.collective_all_reduce_strategy) is deprecated and will be removed in a future version.
Instructions for updating:
use distribute.MultiWorkerMirror

Feel free to stop training early. Check the 'training_progress' folder to see all training checkpoints.


# 5.&nbsp;Convert Model to TFLite

In [15]:
#remove final output folder if it exists
if os.path.exists(FINALOUTPUTFOLDER) and os.path.isdir(FINALOUTPUTFOLDER):
  shutil.rmtree(FINALOUTPUTFOLDER)

# Make a directory to store the trained TFLite model
!mkdir {FINALOUTPUTFOLDER}
print(FINALOUTPUTFOLDER)
# Export graph
# Path to training directory (the conversion script automatically chooses the highest checkpoint file)
last_model_path = HOMEFOLDER+'training_progress'
exporter_path = HOMEFOLDER+'models/research/object_detection/export_tflite_graph_tf2.py'
output_directory = FINALOUTPUTFOLDER

!python $exporter_path \
    --trained_checkpoint_dir $last_model_path \
    --output_directory $output_directory \
    --pipeline_config_path $pipeline_file

# Convert to .tflite Flatbuffer
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model(FINALOUTPUTFOLDER+'/saved_model')
tflite_model = converter.convert()
model_path_32bit = FINALOUTPUTFOLDER+'/limelight_neural_detector_32bit.tflite'
with open(model_path_32bit, 'wb') as f:
  f.write(tflite_model)

!cp {HOMEFOLDER}limelight_neural_detector_labels.txt {FINALOUTPUTFOLDER}
!cp {HOMEFOLDER}models/mymodel/pipeline_file.config {FINALOUTPUTFOLDER}

/kaggle/output
2025-04-01 20:15:44.361633: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-01 20:15:44.361687: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-01 20:15:44.364015: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0401 20:15:51.100952 136492190717056 api.py:460] feature_map_spatial_dims: [(19, 19), (10, 10), (5, 5), (3, 3), (2, 2), (1, 1)]
I0401 20:15:52.599168 136492190717056 convolutional_keras_box_predictor.py:152] depth of additional conv before box predictor: 0
I0401 20:15:52.599447 136492190717056 convolutional_keras_box_predictor.py:152] depth of add

In [16]:
# Export graph
# Path to training directory (the conversion script automatically chooses the highest checkpoint file)
last_model_path = HOMEFOLDER+'training_progress'
exporter_path = HOMEFOLDER+'models/research/object_detection/export_tflite_graph_tf2.py'
output_directory = FINALOUTPUTFOLDER

!python $exporter_path \
    --trained_checkpoint_dir $last_model_path \
    --output_directory $output_directory \
    --pipeline_config_path $pipeline_file

# Convert to .tflite Flatbuffer
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model(FINALOUTPUTFOLDER+'/saved_model')
tflite_model = converter.convert()
model_path_32bit = FINALOUTPUTFOLDER+'/limelight_neural_detector_32bit.tflite'
with open(model_path_32bit, 'wb') as f:
  f.write(tflite_model)

!cp {HOMEFOLDER}limelight_neural_detector_labels.txt {FINALOUTPUTFOLDER}

2025-04-01 20:16:25.933362: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-01 20:16:25.933415: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-01 20:16:25.934788: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0401 20:16:32.517673 131960541856896 api.py:460] feature_map_spatial_dims: [(19, 19), (10, 10), (5, 5), (3, 3), (2, 2), (1, 1)]
I0401 20:16:34.012026 131960541856896 convolutional_keras_box_predictor.py:152] depth of additional conv before box predictor: 0
I0401 20:16:34.012319 131960541856896 convolutional_keras_box_predictor.py:152] depth of additional conv be

# 6. Quantize model
The "TFLiteConverter" module will perform [post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization) on the model. To quantize the model, we need to provide a set of example images. We will extract 100 images from the training tfrecord and place said images into the "extracted_samples" folder.


In [17]:
import tensorflow as tf
import os
import io
from PIL import Image

def extract_images_from_tfrecord(tfrecord_path, output_folder, num_samples=100):
    # Make sure the output directory exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Initialize a counter for the number of images saved
    saved_images = 0

    # Read the TFRecord file
    raw_dataset = tf.data.TFRecordDataset(tfrecord_path)
    for raw_record in raw_dataset.take(num_samples):
        example = tf.train.Example()
        example.ParseFromString(raw_record.numpy())

        # Extract the image data (change 'image/encoded' if necessary)
        image_data = example.features.feature['image/encoded'].bytes_list.value[0]

        # Decode the image data and save as a file
        image = Image.open(io.BytesIO(image_data))
        image.save(os.path.join(output_folder, f'image_{saved_images}.png'))

        saved_images += 1
        if saved_images >= num_samples:
            break

    print(f"Extracted {saved_images} images to {output_folder}")

# Set the path to your TFRecord file and the output directory
tfrecord_path = train_record_fname
extracted_sample_folder = HOMEFOLDER+'extracted_samples'

#remove sample folder if it exists
if os.path.exists(extracted_sample_folder) and os.path.isdir(extracted_sample_folder):
  shutil.rmtree(extracted_sample_folder)

# Extract images
extract_images_from_tfrecord(tfrecord_path, extracted_sample_folder)


# Get list of all images in train directory
from google.cloud import storage
import glob

quant_image_list=[]
if(MLENVIRONMENT=="KAGGLE"):

    jpg_file_list = glob.glob(extracted_sample_folder + '/*.jpg')
    jpeg_file_list = glob.glob(extracted_sample_folder + '/*.jpeg')
    JPG_file_list = glob.glob(extracted_sample_folder + '/*.JPG')
    png_file_list = glob.glob(extracted_sample_folder + '/*.png')
    bmp_file_list = glob.glob(extracted_sample_folder + '/*.bmp')
    quant_image_list = jpg_file_list + JPG_file_list + png_file_list + bmp_file_list

print("pulling samples from " + extracted_sample_folder)
print("samples: " + str(len(quant_image_list)))

Extracted 100 images to /kaggle/working/extracted_samples
pulling samples from /kaggle/working/extracted_samples
samples: 100


In [18]:
# A generator that provides a representative dataset
# Code modified from https://colab.research.google.com/github/google-coral/tutorials/blob/master/retrain_classification_ptq_tf2.ipynb

# First, get input details for model so we know how to preprocess images
interpreter = tf.lite.Interpreter(model_path=model_path_32bit)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]

import random

def representative_data_gen():
  dataset_list = quant_image_list
  quant_num = 300
  for i in range(quant_num):
    pick_me = random.choice(dataset_list)
    print(pick_me)
    image = tf.io.read_file(pick_me)

    if pick_me.endswith('.jpg') or pick_me.endswith('.JPG') or pick_me.endswith('.jpeg'):
      image = tf.io.decode_jpeg(image, channels=3)
    elif pick_me.endswith('.png'):
      image = tf.io.decode_png(image, channels=3)
    elif pick_me.endswith('.bmp'):
      image = tf.io.decode_bmp(image, channels=3)

    image = tf.image.resize(image, [width, height])  # TO DO: Replace 300s with an automatic way of reading network input size
    image = tf.cast(image / 255., tf.float32)
    image = tf.expand_dims(image, 0)
    yield [image]

Finally, we'll initialize the TFLiteConverter module, point it at the TFLite graph we generated in Step 6, and provide it with the representative dataset generator function we created in the previous code block. We'll configure the converter to quantize the model's weight values to INT8 format.

In [19]:
# Initialize converter module
converter = tf.lite.TFLiteConverter.from_saved_model(FINALOUTPUTFOLDER+'/saved_model')
print("initialized converter")
# This enables quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# This sets the representative dataset for quantization
converter.representative_dataset = representative_data_gen
# This ensures that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# For full integer quantization, though supported types defaults to int8 only, we explicitly declare it for clarity.
converter.target_spec.supported_types = [tf.int8]
# These set the input tensors to uint8 and output tensors to float32
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.float32
print("begin conversion")
tflite_model = converter.convert()
print("conversion complete")

with open(FINALOUTPUTFOLDER+'/limelight_neural_detector_8bit.tflite', 'wb') as f:
  f.write(tflite_model)

initialized converter
begin conversion
/kaggle/working/extracted_samples/image_34.png
/kaggle/working/extracted_samples/image_0.png
/kaggle/working/extracted_samples/image_10.png
/kaggle/working/extracted_samples/image_65.png
/kaggle/working/extracted_samples/image_55.png
/kaggle/working/extracted_samples/image_66.png
/kaggle/working/extracted_samples/image_34.png
/kaggle/working/extracted_samples/image_71.png
/kaggle/working/extracted_samples/image_34.png
/kaggle/working/extracted_samples/image_75.png
/kaggle/working/extracted_samples/image_40.png
/kaggle/working/extracted_samples/image_76.png
/kaggle/working/extracted_samples/image_75.png
/kaggle/working/extracted_samples/image_88.png
/kaggle/working/extracted_samples/image_68.png
/kaggle/working/extracted_samples/image_63.png
/kaggle/working/extracted_samples/image_37.png
/kaggle/working/extracted_samples/image_2.png
/kaggle/working/extracted_samples/image_16.png
/kaggle/working/extracted_samples/image_47.png
/kaggle/working/extract

# 7. Compile Model for Limelight & Download


Install Coral Compiler

In [20]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
! sudo apt-get update
! sudo apt-get install edgetpu-compiler

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1022  100  1022    0     0  12552      0 --:--:-- --:--:-- --:--:-- 12617



Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [69.9 kB]
Get:4 https://packages.cloud.google.com/apt coral-edgetpu-stable InRelease [1,423 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,381 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://packages.cloud.go

Compile the previously-generated 8-bit model for Google Coral

In [21]:
!cd {FINALOUTPUTFOLDER} && pwd && edgetpu_compiler limelight_neural_detector_8bit.tflite && pwd && mv limelight_neural_detector_8bit_edgetpu.tflite limelight_neural_detector_coral.tflite && rm limelight_neural_detector_8bit_edgetpu.log

/kaggle/output
Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 1955 ms.

Input model: limelight_neural_detector_8bit.tflite
Input size: 6.42MiB
Output model: limelight_neural_detector_8bit_edgetpu.tflite
Output size: 6.75MiB
On-chip memory used for caching model parameters: 6.52MiB
On-chip memory remaining for caching model parameters: 1.08MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 102
Operation log: limelight_neural_detector_8bit_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 99
Number of operations that will run on CPU: 3
Se

Zip models

In [22]:
!rm {HOMEFOLDER}limelight_detectors.zip
!zip -r {HOMEFOLDER}limelight_detectors.zip /kaggle/output

rm: cannot remove '/kaggle/working/limelight_detectors.zip': No such file or directory
  adding: kaggle/output/ (stored 0%)
  adding: kaggle/output/limelight_neural_detector_32bit.tflite (deflated 7%)
  adding: kaggle/output/limelight_neural_detector_labels.txt (deflated 30%)
  adding: kaggle/output/limelight_neural_detector_8bit.tflite (deflated 17%)
  adding: kaggle/output/limelight_neural_detector_coral.tflite (deflated 21%)
  adding: kaggle/output/saved_model/ (stored 0%)
  adding: kaggle/output/saved_model/assets/ (stored 0%)
  adding: kaggle/output/saved_model/saved_model.pb (deflated 91%)
  adding: kaggle/output/saved_model/variables/ (stored 0%)
  adding: kaggle/output/saved_model/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: kaggle/output/saved_model/variables/variables.index (deflated 77%)
  adding: kaggle/output/saved_model/fingerprint.pb (stored 0%)


Download